#Programatically enabling system tables for Databricks
Since system tables are governed by Unity Catalog, you need at least one Unity Catalog-governed workspace in your account to enable system tables. That way you can map your system tables to the Unity Catalog metastore. System tables must be enabled by an **account admin**. You can enable system tables in your account using the Databricks CLI, the Databricks SDK for Python, or by calling the Unity Catalog API in a notebook.
<br>
<br>
You can enable system tables using API calls or directly in a Databricks notebook (such as this example). Please refer to the documentation for your cloud for further details ([AWS](https://docs.databricks.com/administration-guide/system-tables/index.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/administration-guide/system-tables/))


**Running this notebook will enable the system tables and you will see them in your system catalog.**


<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=governance&org_id=1444828305810485&notebook=%2F_enable_system_tables&demo_name=uc-04-system-tables&event=VIEW&path=%2F_dbdemos%2Fgovernance%2Fuc-04-system-tables%2F_enable_system_tables&version=1&user_hash=ddf348067415027dc6243f8b145629a2bf969356f50e695cd9fcf1cc18a6c40d">

### Creating the widget to be used as input for the metastore ID
The metastore ID can be found by clicking on the metastore details icon present in the `Data` tab of your Databricks workspace.
![metastore_id_image](./images/metastore_id.png)

In [0]:
%pip install databricks-sdk==0.30.0
dbutils.library.restartPython()

In [0]:
metastore_id = spark.sql("SELECT current_metastore() as metastore_id").collect()[0]["metastore_id"]
metastore_id = metastore_id[metastore_id.rfind(':')+1:]
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("metastore_id", metastore_id, "Metastore ID")
metastore_id = dbutils.widgets.get("metastore_id")

### Enabling schemas
Use Python SDK and the workspace client to authenticate, identify available schemas, and enable those schemas

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

system_schemas = w.system_schemas.list(metastore_id=metastore_id)

for system_schema in system_schemas:
    if system_schema.state.value == 'AVAILABLE' and "__internal" not in system_schema.schema:
        print(f'Enabling system schema "{system_schema.schema}"')
        w.system_schemas.enable(metastore_id=metastore_id, schema_name=system_schema.schema)